### Mount my Google Drive

In [ ]:
import os
from google.colab import files, drive   

# mount the google drive to my Colab session
drive.mount('/content/gdrive')
print(os.listdir('/content/gdrive/My Drive/'))

### Install MXNet and GluonCV

In [ ]:
!pip install --upgrade mxnet-cu101 gluoncv

### Import necessary packages

In [ ]:
import mxnet as mx
import numpy as np
import os, time, shutil

from mxnet import gluon, image, init, nd
from mxnet import autograd as ag
from mxnet.gluon import nn
from mxnet.gluon.data.vision import transforms
from gluoncv.utils import makedirs, TrainingHistory
from gluoncv.model_zoo import get_model

In [ ]:
import matplotlib as mpl
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

fs = 20
!wget https://github.com/trishume/OpenTuringCompiler/raw/master/stdlib-sfml/fonts/Times%20New%20Roman.ttf
fm.fontManager.ttflist += fm.createFontList(['Times New Roman.ttf'])
mpl.rc('font', family='Times New Roman', size=fs)

--2021-05-20 22:10:40--  https://github.com/trishume/OpenTuringCompiler/raw/master/stdlib-sfml/fonts/Times%20New%20Roman.ttf
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/trishume/OpenTuringCompiler/master/stdlib-sfml/fonts/Times%20New%20Roman.ttf [following]
--2021-05-20 22:10:41--  https://raw.githubusercontent.com/trishume/OpenTuringCompiler/master/stdlib-sfml/fonts/Times%20New%20Roman.ttf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 834452 (815K) [application/octet-stream]
Saving to: ‘Times New Roman.ttf’

Times New Roman.ttf 100%[===================>] 814.89K  --.-KB/s    in 0.06s   



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: MatplotlibDeprecationWarning: 
The createFontList function was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use FontManager.addfont instead.
  import sys


### Data preparation & Data augmentation

In [ ]:
path = '/content/gdrive/My Drive/embryo-89-8cell-DIC_AIF_DTCWT-onset'

train_path = os.path.join(path, 'train')
test_path = os.path.join(path, 'validate')

In [ ]:
npzfile = np.load(os.path.join(path, 'train_mean_std.npz'))
X_mean = npzfile['X_mean'].tolist()
X_std = npzfile['X_std'].tolist()
print(X_mean,X_std)

128.17645593242332 42.19776721563477


In [ ]:
# z-score standardize the data
transform_train = transforms.Compose([
    transforms.Resize(600, keep_ratio=True),
    transforms.CenterCrop(512),

    transforms.RandomFlipLeftRight(), # Randomly flip the image horizontally
    transforms.RandomFlipTopBottom(),
    transforms.ToTensor(),
    transforms.RandomRotation((-15, 15),rotate_with_proba=0.5),
    transforms.Normalize([X_mean], [X_std])
])

transform_test = transforms.Compose([
    transforms.Resize(600, keep_ratio=True),
    transforms.CenterCrop(512),

    transforms.ToTensor(),
    transforms.Normalize([X_mean], [X_std])
])

### Set hyperparameters 

In [ ]:
classes = 2

per_device_batch_size = 16
num_gpus = 1
num_workers = 8
ctx = [mx.gpu(i) for i in range(num_gpus)] if num_gpus > 0 else [mx.cpu()]
batch_size = per_device_batch_size * max(num_gpus, 1)

### Define data loader

In [ ]:
train_data = gluon.data.DataLoader(
    gluon.data.vision.ImageFolderDataset(train_path).transform_first(transform_train),
    batch_size=batch_size, shuffle=True, num_workers=num_workers)

test_data = gluon.data.DataLoader(
    gluon.data.vision.ImageFolderDataset(test_path).transform_first(transform_test),
    batch_size=batch_size, shuffle=False, num_workers = num_workers)

### Optimizer, Loss and Metric

In [ ]:
lr = 1e-3
lr_factor = 0.75 # Learning rate decay factor
lr_steps = [10, 20, 30, np.inf] # Epochs where learning rate decays
wd = 0.0001
momentum = 0.9

# optimizer = 'sgd' # 'nag': Nesterov accelerated gradient descent
# optimizer_params = {'learning_rate': lr, 'wd': wd, 'momentum': momentum} # Set parameters

optimizer = 'adam'
optimizer_params = {'learning_rate': lr} # Set parameters

L = gluon.loss.SoftmaxCrossEntropyLoss()

metric = mx.metric.Accuracy()
train_history = TrainingHistory(['training-acc', 'validation-acc'])

In [ ]:
# define an evaluation function for validation and testing
def test(net, test_data, ctx):
    metric = mx.metric.Accuracy()
    for i, batch in enumerate(test_data):
        data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0, even_split=False)
        label = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0, even_split=False)
        outputs = [net(X) for X in data]
        metric.update(label, outputs)

    return metric.get()

### Training

In [ ]:
for idx in range(1,4,1):
  epochs = 40
  lr_counter = 0
  num_batch = len(train_data)
  train_acc_list = []
  val_acc_list = []

  model_name = 'ResNet50_v2'
  # Fine-tuning pre-trained models: train model on my new data using the pre-trained weights as initialization -- transfer learning
  finetune_net = get_model(model_name, pretrained=True)
  with finetune_net.name_scope():
      finetune_net.output = nn.Dense(classes)

  finetune_net.output.initialize(init.Xavier(), ctx = ctx)
  finetune_net.collect_params().reset_ctx(ctx)
  finetune_net.hybridize()

  trainer = gluon.Trainer(finetune_net.collect_params(), optimizer, optimizer_params)

  # training loop
  for epoch in range(epochs):
      if epoch == lr_steps[lr_counter]:
          trainer.set_learning_rate(trainer.learning_rate*lr_factor)
          lr_counter += 1

      tic = time.time()
      train_loss = 0
      metric.reset()

      for i, batch in enumerate(train_data):
          data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0, even_split=False)
          label = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0, even_split=False)
          with ag.record():
              outputs = [finetune_net(X) for X in data]
              loss = [L(yhat, y) for yhat, y in zip(outputs, label)]
          for l in loss:
              l.backward()

          trainer.step(batch_size)
          train_loss += sum([l.mean().asscalar() for l in loss]) / len(loss)

          metric.update(label, outputs)

      _, train_acc = metric.get()
      train_loss /= num_batch

      _, val_acc = test(finetune_net, test_data, ctx)
      train_history.update([train_acc, val_acc])
      train_acc_list.append(train_acc)
      val_acc_list.append(val_acc)

      print('[Epoch %d] Train-acc: %.3f, loss: %.3f | Val-acc: %.3f | time: %.1f' %
              (epoch, train_acc, train_loss, val_acc, time.time() - tic))

  train_history.plot(['training-acc', 'validation-acc'])
  # save model parameters
  model_file_name = '/content/gdrive/My Drive/ResNet50_v2_weights_DIC_AIF_DTCWT/trial'+str(idx)+'.params'
  finetune_net.save_parameters(model_file_name)